In [1]:
import inspect

from src.llm_plan.planner import Planner
from src.llm_plan.llm import ChatGPT

# Human specific
specific = inspect.cleandoc("""You are visiting San Francisco for the day and want to meet as many friends as possible. 
                           Solve the problem by considering various different schedules and picking the best one to optimize your goals. 
                           Travel distances (in minutes): 
                           
                           - Marina District to Alamo Square: 15. 
                           - Alamo Square to Marina District: 15. 
                           
                           Constraints:
                           You arrive at Marina District at 9:00AM. 
                           James will be at Alamo Square from 3:45PM to 8:00PM. 
                           
                           Goal:
                           You'd like to meet James for a minimum of 75 minutes.""")
environment_name = "MeetingPlanning"

In [3]:
# Generate the json representation of the environment
model = ChatGPT("o4-mini")
planner = Planner(model)

planner.generate_representation(specific, environment_name ,"json")

Here's an example of a json file that describes an environment:
                                 <environment-json>{'name': 'TwoAgentsVault', 'author': 'Human', 'agents': {'number': 3, 'names': ['key_holder', 'small_robot', 'orchestrator'], 'key_holder': {'private_information': ['I have a key', 'I am a big robot'], 'goal': 'Open the vault and grab the object inside.'}, 'small_robot': {'private_information': ['I am a small robot'], 'goal': 'Open the vault and grab the object inside.'}, 'orchestrator': {'private_information': [], 'goal': 'Open the vault and grab the object inside.'}}, 'environment': {'init': {'grid_size': 4, 'visibility': 1}, 'public_information': ['There is a vault in the environment.', 'The vault is closed.', 'The vault requires a key to be opened.', 'The entrance of the vault is small.']}, 'workflow': {'key_holder': {'pddl': {'input': [], 'output': 'pddl_key_holder', 'system_prompt': 'You are an expert with PDDL problems (Planning Domain Definition Language). You alwa

PosixPath('environments/static/MeetingPlanning')

In [5]:
# Generate the environment plan
from src.llm_plan.environment import Environment

env = Environment(f"./environments/static/{environment_name}.json")
env.plan

[['planner.pddl']]

In [6]:
# Plan
responses = planner.plan(env)

----- planner->pddl -----

----- System Prompt -----
You are an expert in temporal planning with PDDL. You always provide a PDDL domain and a PDDL problem file to model scheduling with time windows.

----- Prompt -----
Your name is planner. You are in an environment with the following public information:['Travel time between Marina District and Alamo Square is 15 minutes', 'You arrive at Marina District at 09:00', 'James will be at Alamo Square from 15:45 to 20:00']You have the following private information:['I arrive at Marina District at 09:00', 'I want to meet James for at least 75 minutes']Your goal is:Meet James at Alamo Square for a minimum of 75 minutesThink step by step and provide a PDDL domain and a PDDL problem file to solve the scheduling problem. Enclose the domain in <domain></domain> tags and the problem in <problem></problem> tags.

----- Response -----
<domain>
(define (domain meeting-scheduling)
  (:requirements :typing :durative-actions :fluents :timed-initial-litera

In [8]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

# TODO: fix the vocabulary entry, it shouldn't be "pddl_scheduler" etc.
final_plan = responses["pddl_planner"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [9]:
# Obtain the plan with the solver
import subprocess

from pathlib import Path

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

BASE_FOLDER = Path(f"./tmp/{env.name}")
BASE_FOLDER.mkdir(parents=True, exist_ok=True)

with open(BASE_FOLDER / "problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "sas_plan",
    BASE_FOLDER / "domain.pddl",
    BASE_FOLDER / "problem.pddl",
]

with open(BASE_FOLDER / "logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)

In [10]:
# Validate the plan with uVAL
import subprocess

from src.llm_plan.config import UNIVERSAL_VALIDATOR_BIN

command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'domain.pddl'} \
{BASE_FOLDER / 'problem.pddl'} \
{BASE_FOLDER / 'sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: TIMED-INITIAL-LITERALS does not name a known token



In [11]:
# Let's try to refine the plan syntax
from src.llm_plan.hypervisor import HypervisorDeepThinkPDDL, HypervisorSyntaxPDDL

num_attempts = 10

for i in range(num_attempts):
    
    # General validator
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
    }

    deep_think = HypervisorDeepThinkPDDL(llm=model, prompt_args=prompt_args)

    r = deep_think.run()
    domain, problem = pddl_parser.parse(r, from_file=False)
    
    # Obtain the new plan with the solver
    with open(BASE_FOLDER / "refined_problem.pddl", "w") as f:
        f.write(problem)

    with open(BASE_FOLDER / "refined_domain.pddl", "w") as f:
        f.write(domain)
        
    # Launch the solver
    command = [
        SOLVER_BINARY,
        *SOLVER_ARGS,
        BASE_FOLDER / "refined_sas_plan",
        BASE_FOLDER / "refined_domain.pddl",
        BASE_FOLDER / "refined_problem.pddl",
    ]

    with open(BASE_FOLDER / "refined_logs.txt", "w") as logfile:
        subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
        
    command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
    {BASE_FOLDER / 'refined_domain.pddl'} \
    {BASE_FOLDER / 'refined_problem.pddl'} \
    {BASE_FOLDER / 'refined_sas_plan'}"

    out = subprocess.run(command, shell=True, capture_output=True, text=True)

    # This part won't be printed at test time
    if out.stderr:
        pddl_error = out.stderr
        print(
            f"The validation found a problem with the plan: {out.stderr}"
        )
    else:
        pddl_error = "No error was found."
        print("The plan is valid.")
        print("[stdout]", out.stdout)
    
    # Syntax validator
    print(f"Validation attempt {i+1}/{num_attempts}")
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
        "syntax_errors": pddl_error,
    }

    hypervisor_pddl = HypervisorSyntaxPDDL(llm=model, prompt_args=prompt_args)

    r = hypervisor_pddl.run()
    domain, problem = pddl_parser.parse(r, from_file=False)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: DURABLE-ACTIONS does not name a known token

Validation attempt 1/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: TIMED-INITIAL-LITERALS does not name a known token

Validation attempt 2/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: TIMED-INITIAL-LITERALS does not name a known token

Validation attempt 3/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: DURATION-INEQUALITIES does not name a known token

Validation attempt 4/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: DURABLE-ACTIONS does not name a known token

Vali

In [15]:
# Final plan run and validation
with open(BASE_FOLDER / "refined_problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "refined_domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "refined_sas_plan",
    BASE_FOLDER / "refined_domain.pddl",
    BASE_FOLDER / "refined_problem.pddl",
]

with open(BASE_FOLDER / "refined_logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
    
command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'refined_domain.pddl'} \
{BASE_FOLDER / 'refined_problem.pddl'} \
{BASE_FOLDER / 'refined_sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: 405 does not name a known token



In [14]:
# Report the natural language plan
from src.llm_plan.hypervisor import HypervisorNaturalLanguage

with open(BASE_FOLDER / "refined_problem.pddl", "r") as f:
    problem = f.read()

with open(BASE_FOLDER / "refined_domain.pddl", "r") as f:
    domain = f.read()
    
with open(BASE_FOLDER / "refined_sas_plan", "r") as f:
    plan = f.read()
    
prompt_args = {
    "specification": env.config_data,
    "pddl_domain": domain,
    "pddl_problem": problem,
    "pddl_plan": plan,
}

hypervisor_to_nl = HypervisorNaturalLanguage(llm=model, prompt_args=prompt_args)

natural_plan = hypervisor_to_nl.run()

with open(BASE_FOLDER / "refined_nl_plan.txt", "w") as f:
    f.write(natural_plan)

FileNotFoundError: [Errno 2] No such file or directory: 'tmp/MeetJamesSchedule/refined_sas_plan'

In [16]:
import inspect

prompt = inspect.cleandoc("""
    The following prompt expresses a request in natural language.
    {natural_prompt}

    This is the SQL schema of the database.
    {sql_schema}

    You have to turn the natural prompt into an SQL query that is:
    - Compatible with the SQL schema.
    - Safe against SQL injections and vulnerabilities.

    Think step by step and provide the SQL query between <SQL></SQL> tags.
""")

print(prompt)

The following prompt expresses a request in natural language.
{natural_prompt}

This is the SQL schema of the database.
{sql_schema}

You have to turn the natural prompt into an SQL query that is:
- Compatible with the SQL schema.
- Safe against SQL injections and vulnerabilities.

Think step by step and provide the SQL query between <SQL></SQL> tags.
